In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle


In [3]:
## load the model

model=load_model('model.h5')

##load the encoder and scaler
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)



In [5]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [9]:
## one hot encoding

oe_encoded=onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
oe_geo=pd.DataFrame(oe_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
oe_geo

c:\Users\ACER\OneDrive\Desktop\DS\Python\ANN_classification\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [11]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [12]:
input_data=pd.concat([input_df.reset_index(drop=True),oe_geo],axis=1)

input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [13]:
input_data['Gender']=label_encoder_gender.transform(input_data['Gender'])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [14]:
#drop the Geography column
input_data.drop(['Geography'],axis=1,inplace=True)

In [15]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [16]:
## Standard scaler

input_scaled=scaler.transform(input_data)
input_scaled

array([[-0.54204762,  0.91055421,  0.10473081, -0.68894811, -0.26230046,
         0.81966266,  0.64598061,  0.97071435, -0.88153859,  0.99828718,
        -0.57559072, -0.57779016]])

In [17]:
##predictiom

pred=model.predict(input_scaled)
pred

1/1 [==============================] - 0s 360ms/step


array([[0.02265117]], dtype=float32)

In [18]:
pred_prob=pred[0][0]
pred_prob

0.022651171

In [19]:
if pred_prob>0.5:
    print('The Customer is likely to churn')
else:
    print('The Customer is not likely to churn')

The Customer is not likely to churn
